## Install fastai=0.0.7

In [ ]:
!git clone https://github.com/fastai/fastai1.git

In [ ]:
pwd

In [ ]:
cd fastai1/old/

In [ ]:
pip install .

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline


In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
cd '/kaggle/working'

In [ ]:
PATH = "../input/bluebook-for-bulldozers/"

In [ ]:
!ls {PATH}

In [ ]:
!unzip "../input/bluebook-for-bulldozers/Train.zip" -d "../input/bluebook-for-bulldozers/Train.csv"

In [ ]:
!ls {PATH}

In [ ]:
df_raw = pd.read_csv('Train.csv', low_memory=False, parse_dates=["saledate"])

### EDA

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

In [ ]:
display_all(df_raw.tail().T)

In [ ]:
display_all(df_raw.describe(include='all').T)

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [ ]:
df_raw.SalePrice

### Inital Processing


In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df_raw.drop("SalePrice", axis=1), df_raw.SalePrice)

### use fastai to create extract features from date

In [ ]:
add_datepart(df_raw, "saledate")

In [ ]:
df_raw.head()

In [ ]:
df_raw.saleYear.head()

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.



In [ ]:
train_cats(df_raw)

We can specify the order to use for categorical variables if we wish:



In [ ]:
df_raw.UsageBand.cat.categories

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.



In [ ]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes

We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.

In [ ]:
display_all(df_raw.isnull().sum().sort_index() / len(df_raw))

But let's save this file for now, since it's already in format can we be stored and accessed efficiently.


In [ ]:
os.makedirs("tmp", exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

### Pre-processing


In [ ]:
df_raw = pd.read_feather("tmp/bulldozers-raw")

In [ ]:
?proc_df

In [ ]:
df, y , nas = proc_df(df_raw, "SalePrice")

In [ ]:
display_all(df.head().T)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df, y)

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape


## Random forests


### Base Model


In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)